Importing packages

In [ ]:
%matplotlib qt5

import numpy as np
import matplotlib.pyplot as plt
import pymodels as pm
from siriuspy.devices import PowerSupplyPU, PowerSupply, Trigger, FamBPMs
from apsuite.commisslib.meas_bo_tune_by_bpm import BOTunebyBPMParams, BOTunebyBPM

# Preparatives

## Reducing the BO extraction kicker delay in 10 ms. 

In [ ]:
ejekckr = PowerSupplyPU(PowerSupplyPU.DEVICES.BO_EJE_KCKR)
# ejekckr_delay0 = ejekckr.delay
# ejekckr.delay = ejekckr_delay0 - 10000 

## Configure BPMs to start storing data at the same time of the kicker pulse and set the number of points

In [ ]:
params = BOTunebyBPMParams()
params.event = 'DigBO'           # |Must be the same as ejekckr
params.trigger_source = 'InjBO'  # |
params.trigger_source_mode = 'Disabled'
params.extra_delay = ejekckr.delay  # Same delay as ejekckr
params.nr_pulses = 1
params.nr_points_after = 6035
params.nr_points_before = 500
params.bpms_timeout = 30  # [s]

In [ ]:
measobj = BOTunebyBPM(params=params, isonline=False)
measobj.configure_bpms()

## Approaching  tunes

**Make this part in BO control screen:**

Approach tunes until $\Delta \approx 0.025.$

## Inject (with the kicker on) and get the test data

In [ ]:
measobj.get_orbit(injection=False, external_trigger=False)

In [ ]:
measobj.save_pickle(fname = 'data_for_test')

## Checking orbit and tunes

Looking at the oscillations

In [ ]:
data = measobj.data
orbx0, orby0 = data['orbx'][:, 0], data['orby'][:, 0]
x = orbx0 - orbx0.mean(axis=0)
y = orby0 - orby0.mean(axis=0)

In [ ]:
ax,fig = plt.subplots(2,1, sharex=True)
fig.set_suptitle("Betatron Oscillations ")
ax[0].plot(x)
ax[1].plot(y)
ax[1].set_xlabel('Turns')
ax[0].set_ylabel('x')
ax[1].set_ylabel('y')

Checking the Tunes

In [ ]:
spectrumx, spectrumy, freqs = measobj.dft(bpm_indices=[0])
plt.plot(spectrumx)

In [ ]:
tune1, tune2 = measobj.naff_tunes(self, dn=200, window_param=1, bpm_indices=None)
plt.plot(tune1)
plt.plot(tune2)
plt.title('Tune evolution')
plt.xlabel('Turns')
pĺt.ylabel(r'$\nu$')
delta = tune1 - tune2
print(f'Delta = {delta}')

In [ ]:
tune1_matrix, tune2_matrix, freqs, revs = spectrogram(
    self, dn=None, overlap=True, bpm_indices=None)

# Measure tunes while changing quadrupoles force

At each iteration, we must change the tunes in $\approx 0.002$, in direction to resonance. This corresponds to a change $ dKL \approx 1.75 e-05$ in quadrupole integrated forces. 

If $\nu_x > \nu_y$, the change must be positive in QD or negative in QF. The opposite follows for the case  $\nu_x < \nu_y$.

In [ ]:
# Iteration counter
i = 0 

**For 25 iterations:**

 1 - In BO control screen, make variation of $d \Delta \approx 0.002$

2 - Run the following lines to do the measure and check the result

In [ ]:
measobj.get_orbit()

In [ ]:
*_ = spectrogram(
    self, dn=None, overlap=True, bpm_indices=[0, 1 ,2, 3, 4, 5])

tune1, tune2 = measobj.naff_tunes(self, dn=None, window_param=1, bpm_indices=None)
print(f'tune1 = {tune1} \n tune2 = {tune2} \n delta = {tune1-tune2}')

3 -Input the tune separation on the method above to store the devices data:

In [ ]:
delta = None
measobj.get_data(delta=delta)

4 - Save the data dictionary

In [ ]:
deltastr = str(measobj.data['delta'])
measobj.save_data(fname=deltastr, overwrite=False)